<a href="https://colab.research.google.com/github/perch97/dl_hse/blob/main/linear_models_old_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb
from google.colab import drive
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error


In [5]:
 import matplotlib.pyplot as plt
import pandas as pd
import os
import joblib
import numpy as np

In [6]:
from sklearn.model_selection import GridSearchCV


In [7]:
relaxed = pd.read_csv('/content/drive/MyDrive/prop_df_gmp_relaxed_scaled.csv')
relaxed.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
unrelaxed = pd.read_csv('/content/drive/MyDrive/prop_df_with_gmp_features_scaled1.csv')

In [9]:

unrelaxed.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
unrelaxed.head()

,FNN0,FNN1,FNN2,FNN3,FNN4,FNN5,FNN6,FNN7,FNN8,FNN9,...,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,0.091630,-0.265687,-1.157498,-0.035791,0.047572,1.168317,-0.496529,-0.695215,-0.737316,-0.749878,...,1.843369,1.840496,-1.099231,-1.635341,-1.572575,-0.266095,-1.149051,-1.512225,-1.344746,-1.051016
1,-0.756939,1.975220,0.211758,-2.269964,-0.272965,-2.221339,0.380603,-1.071306,-0.453924,-2.261901,...,0.227599,0.690825,-1.109893,-0.874160,-0.973754,-1.392098,-1.095579,-0.956363,-0.578942,-0.025752
2,1.881340,-0.348301,0.668177,0.697942,1.650257,-0.297481,-0.034444,1.605270,1.832090,0.762145,...,-1.359470,-0.961115,0.626351,0.743491,0.779786,1.999722,1.336005,0.863266,0.643215,0.386986
3,1.881340,-0.348301,0.668177,0.697942,1.650257,-0.297481,-0.034444,1.605270,1.832090,0.762145,...,-2.257170,0.203032,1.766632,1.661302,1.616373,-0.892621,-0.491647,-0.979577,-0.467345,-0.252505
4,-0.340369,-0.007518,0.211758,-0.044035,-0.272965,0.252193,2.040790,0.240616,-0.112191,0.762145,...,-0.294055,-0.643635,0.196149,0.311901,0.262203,-0.048651,-0.135382,0.065000,-0.134564,-0.189781


In [12]:
relaxed

,FNN0,FNN1,FNN2,FNN3,FNN4,FNN5,FNN6,FNN7,FNN8,FNN9,...,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,0.114727,-0.258643,-1.150577,-0.046816,0.070327,1.160169,-0.503675,-0.683833,-0.732208,-0.739943,...,1.843369,1.840496,-1.099231,-1.635341,-1.572575,-0.266095,-1.149051,-1.512225,-1.344746,-1.051016
1,-0.748788,1.981555,0.230380,-2.279193,-0.256165,-2.251003,0.373625,-1.059361,-0.450645,-2.271682,...,0.227599,0.690825,-1.109893,-0.874160,-0.973754,-1.392098,-1.095579,-0.956363,-0.578942,-0.025752
2,1.935960,-0.341231,0.690699,0.686326,1.702784,-0.314933,-0.041501,1.613212,1.820617,0.791796,...,-1.359470,-0.961115,0.626351,0.743491,0.779786,1.999722,1.336005,0.863266,0.643215,0.386986
3,1.935960,-0.341231,0.690699,0.686326,1.702784,-0.314933,-0.041501,1.613212,1.820617,0.791796,...,-2.257170,0.203032,1.766632,1.661302,1.616373,-0.892621,-0.491647,-0.979577,-0.467345,-0.252505
4,-0.324881,-0.000556,0.230380,-0.055054,-0.256165,0.238230,2.034128,0.250599,-0.111117,0.791796,...,-0.294055,-0.643635,0.196149,0.311901,0.262203,-0.048651,-0.135382,0.065000,-0.134564,-0.189781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20765,0.522935,1.651204,-0.229939,0.019084,0.723310,0.238230,2.283203,-0.443870,-0.282916,0.791796,...,0.809197,0.291466,-0.770682,-0.432960,-0.312524,0.523167,0.373019,0.733803,0.933895,0.904008
20766,-0.654586,0.195591,1.151018,-0.944710,-0.256165,-1.697840,-1.104223,0.577408,0.704566,0.791796,...,-0.135803,1.591315,1.493759,0.901732,0.656780,-4.258891,-0.958355,-0.911362,-1.251341,-1.107587
20767,-1.078494,-0.898700,0.690699,-0.771721,-1.235639,0.791393,0.664213,-0.119784,-0.704526,-0.739943,...,0.493974,0.539624,-1.072871,-0.389133,-0.161446,0.312084,-0.695028,-0.689550,0.069441,0.243948
20768,1.888859,-0.911088,1.151018,0.686326,1.702784,-0.591514,-0.556257,2.143369,2.613601,0.791796,...,-1.601032,-0.188755,1.347675,0.791184,0.492950,1.441261,2.253664,1.561141,0.025356,-1.211907


In [13]:
relaxed['name'] = relaxed.index
unrelaxed['name'] = unrelaxed.index

In [14]:
target = pd.read_csv('/content/drive/MyDrive/id_prop_filtered_17000.csv')

In [15]:
target_c = pd.read_csv('/content/drive/MyDrive/id_prop_filtered_17000.csv')

In [16]:
target.rename(columns = {'-0.01887750000000743':'target'}, inplace =True)
target_c.rename(columns = {'-0.01887750000000743':'target'}, inplace =True)

In [17]:
target_c.drop('Unnamed: 0', axis=1, inplace=True)
target.drop('Unnamed: 0', axis=1, inplace=True)

In [18]:
target_c

,name,target
0,0,-0.018878
1,1,-0.654593
2,3,-0.960414
3,4,-0.326443
4,5,-0.286225
...,...,...
17155,20762,-0.018823
17156,20764,-1.365925
17157,20765,-0.079967
17158,20767,-0.321586


In [19]:
relaxed = pd.merge(relaxed,target,on='name')
relaxed.drop(['name','target'], axis=1, inplace=True)

In [20]:
unrelaxed = pd.merge(unrelaxed,target_c,on='name')
unrelaxed.drop(['name','target'], axis=1, inplace=True)

In [21]:
unrelaxed.head()

,FNN0,FNN1,FNN2,FNN3,FNN4,FNN5,FNN6,FNN7,FNN8,FNN9,...,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,0.091630,-0.265687,-1.157498,-0.035791,0.047572,1.168317,-0.496529,-0.695215,-0.737316,-0.749878,...,1.843369,1.840496,-1.099231,-1.635341,-1.572575,-0.266095,-1.149051,-1.512225,-1.344746,-1.051016
1,-0.756939,1.975220,0.211758,-2.269964,-0.272965,-2.221339,0.380603,-1.071306,-0.453924,-2.261901,...,0.227599,0.690825,-1.109893,-0.874160,-0.973754,-1.392098,-1.095579,-0.956363,-0.578942,-0.025752
2,1.881340,-0.348301,0.668177,0.697942,1.650257,-0.297481,-0.034444,1.605270,1.832090,0.762145,...,-2.257170,0.203032,1.766632,1.661302,1.616373,-0.892621,-0.491647,-0.979577,-0.467345,-0.252505
3,-0.340369,-0.007518,0.211758,-0.044035,-0.272965,0.252193,2.040790,0.240616,-0.112191,0.762145,...,-0.294055,-0.643635,0.196149,0.311901,0.262203,-0.048651,-0.135382,0.065000,-0.134564,-0.189781
4,-0.432940,1.582803,-0.701079,-0.353192,-0.272965,-0.297481,-0.447333,-1.434517,-0.890998,-1.505889,...,1.359432,1.298984,-1.065099,-0.746605,-0.715401,-0.139018,-1.045332,-0.714778,-0.209055,-0.013544


In [22]:
target =target['target']
target

,target
0,-0.018878
1,-0.654593
2,-0.960414
3,-0.326443
4,-0.286225
...,...
17155,-0.018823
17156,-1.365925
17157,-0.079967
17158,-0.321586


In [24]:
target

,target
0,-0.018878
1,-0.654593
2,-0.960414
3,-0.326443
4,-0.286225
...,...
17155,-0.018823
17156,-1.365925
17157,-0.079967
17158,-0.321586


In [10]:
#xgboost_0_rel = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=200, alpha = 0.1, reg_lambda = 0.5, learning_rate=0.1, gamma=0, subsample=0.6,
#                          colsample_bytree=1, max_depth=8)

In [28]:
#xgboost_0_rel.fit(relaxed.head(14000), target_c['target'].head(14000))

In [30]:
#target_c

In [31]:
#r2_score(target_c['target'].tail(3160), xgboost_0_rel.predict(unrelaxed.tail(3160)))

In [32]:
#r2_score(target_c['target'].head(14000), xgboost_0_rel.predict(relaxed.head(14000)))

In [33]:
#mean_absolute_error(target_c['target'].head(14000), xgboost_0_rel.predict(relaxed.head(14000)))

In [34]:
#mean_absolute_error(target_c['target'].tail(3160), xgboost_0_rel.predict(unrelaxed.tail(3160)))

In [35]:
#target_c

In [ ]:
# def train_model(model_name, model, X_train,y_train, params,save_dir):
#     print(f"Training {model_name}...")
#     if params:
#         # Use GridSearchCV for hyperparameter tuning
#         grid_search = GridSearchCV(model, param_grid=params, cv=5, scoring='r2', verbose=1)
#         grid_search.fit(X_train, y_train)
#         best_model = grid_search.best_estimator_
#         print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
#     else:
#         model.fit(X_train, y_train)
#         best_model = model
#     # Save the trained model
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)
#     model_filename = os.path.join(save_dir, f"{model_name}_model.joblib")
#     joblib.dump(best_model, model_filename)
#     print(f"Saved {model_name} model to {model_filename}")
#     return best_model



In [ ]:
# X_train = relaxed.head(17000).to_numpy()
# y_train = target.head(17000).to_numpy()
# X_test = unrelaxed.head(17000).to_numpy()
# y_test = target.head(17000).to_numpy()

In [ ]:
# relaxed.head()

,FNN0,FNN1,FNN2,FNN3,FNN4,FNN5,FNN6,FNN7,FNN8,FNN9,...,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15
0,0.114727,-0.258643,-1.150577,-0.046816,0.070327,1.160169,-0.503675,-0.683833,-0.732208,-0.739943,...,1.843369,1.840496,-1.099231,-1.635341,-1.572575,-0.266095,-1.149051,-1.512225,-1.344746,-1.051016
1,-0.748788,1.981555,0.230380,-2.279193,-0.256165,-2.251003,0.373625,-1.059361,-0.450645,-2.271682,...,0.227599,0.690825,-1.109893,-0.874160,-0.973754,-1.392098,-1.095579,-0.956363,-0.578942,-0.025752
2,1.935960,-0.341231,0.690699,0.686326,1.702784,-0.314933,-0.041501,1.613212,1.820617,0.791796,...,-1.359470,-0.961115,0.626351,0.743491,0.779786,1.999722,1.336005,0.863266,0.643215,0.386986
3,1.935960,-0.341231,0.690699,0.686326,1.702784,-0.314933,-0.041501,1.613212,1.820617,0.791796,...,-2.257170,0.203032,1.766632,1.661302,1.616373,-0.892621,-0.491647,-0.979577,-0.467345,-0.252505
4,-0.324881,-0.000556,0.230380,-0.055054,-0.256165,0.238230,2.034128,0.250599,-0.111117,0.791796,...,-0.294055,-0.643635,0.196149,0.311901,0.262203,-0.048651,-0.135382,0.065000,-0.134564,-0.189781


In [ ]:
import itertools
import os
import joblib
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
# Uncomment or remove the following import depending on whether you want to use GPR
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import Matern

def generate_param_combinations(param_grid):
    """
    Generates all possible combinations of parameters from a dictionary.
    """
    keys, values = zip(*param_grid.items())
    return [dict(zip(keys, v)) for v in itertools.product(*values)]

models = {
    'linear_regression': LinearRegression(),
    'GBR': GradientBoostingRegressor(),
    'KRR': KernelRidge(),
    'ELN': ElasticNet(),
    'SVR': SVR(),
    # Uncomment if using GPR
    # 'GPR': GaussianProcessRegressor(kernel=Matern(length_scale=1.0, nu=2.5)),
    'ETR': ExtraTreesRegressor(),
    'XGB': XGBRegressor()
}

# Define hyperparameters for each model
model_hyperparameters = {
    'linear_regression': [{}],
    'GBR': generate_param_combinations({
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05],
        'max_depth': [3, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }),
    'KRR': generate_param_combinations({
        'alpha': [1.0, 0.5, 0.1],
        'kernel': [ 'rbf', 'polynomial'],
        'degree': [1, 2]
    }),
    'ELN': generate_param_combinations({
        'alpha': [0.01, 0.1],
        'l1_ratio': [0.5, 0.7]
    }),
    'SVR': generate_param_combinations({
        'C': [1.0, 10.0],
        'epsilon': [0.1, 0.01],
        'kernel': ['rbf', 'linear']
    }),
    # Uncomment if using GPR
    # 'GPR': generate_param_combinations({
    #     'kernel': [Matern(length_scale=1.0, nu=2.5), Matern(length_scale=0.5, nu=1.5)]
    # }),
    'ETR': generate_param_combinations({
        'n_estimators': [100, 200],
        'max_features': [0.75, 'sqrt']
    }),
    'XGB': generate_param_combinations({
        'n_estimators': [100, 200, 1000],
        'max_depth': [3, 4, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.6, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    })
}

def train_model(model_name, model, X_train, y_train, params, save_dir):
    print(f"Training {model_name}...")
    if params and isinstance(params, list):
        # Convert list of dictionaries into a single dictionary of lists
        combined_params = {key: list(set([d[key] for d in params])) for key in params[0].keys()}

        # Use GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(model, param_grid=combined_params, cv=5, scoring='r2', verbose=1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
    else:
        model.fit(X_train, y_train)
        best_model = model

    # Save the trained model
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    model_filename = os.path.join(save_dir, f"{model_name}_model.joblib")
    joblib.dump(best_model, model_filename)
    print(f"Saved {model_name} model to {model_filename}")

    return best_model


# Sample training data placeholders (should be replaced with actual data)
X_train = relaxed.head(17000).to_numpy()
y_train = target.head(17000).to_numpy()
X_test = unrelaxed.to_numpy()
y_test = target_c.to_numpy()

save_dir = '/content/drive/MyDrive/all_linear_models_gmp_old'
trained_models = {}

for model_name, model in models.items():
    params = model_hyperparameters.get(model_name, [{}])
    trained_models[model_name] = train_model(model_name, model, X_train, y_train, params, save_dir)


Training linear_regression...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best hyperparameters for linear_regression: {}
Saved linear_regression model to /content/drive/MyDrive/all_linear_models_gmp_old/linear_regression_model.joblib
Training GBR...
Fitting 5 folds for each of 32 candidates, totalling 160 fits


In [2]:
from sklearn.metrics import mean_absolute_error

# Assuming 'trained_models' is a dictionary of trained models
# And 'df_gmp_relaxed_scaled' is your feature data and 'target' your target variable
print('MAE on TRAIN DATA:')
for model_name, model in trained_models.items():
  predictions = model.predict(X_train) # Generate predictions
  mae = mean_absolute_error(y_train, predictions) #Calculate MAE
  print(f'{model_name}: MAE = {mae}') #Print MAE for each model

NameError: name 'relaxed' is not defined

In [ ]:
from sklearn.metrics import mean_absolute_error

# Assuming 'trained_models' is a dictionary of trained models
# And 'df_gmp_relaxed_scaled' is your feature data and 'target' your target variable
print('MAE on TEST DATA:')
for model_name, model in trained_models.items():
  predictions = model.predict(X_test) # Generate predictions
  mae = mean_absolute_error(y_test, predictions) #Calculate MAE
  print(f'{model_name}: MAE = {mae}') #Print MAE for each model

In [ ]:
from sklearn.metrics import r2_score

# Assuming 'trained_models' is a dictionary of trained models
# And 'df_gmp_relaxed_scaled' is your feature data and 'target' your target variable

for model_name, model in trained_models.items():
  predictions = model.predict(relaxed.head(17000)) #Generate predictions
  r2 = r2_score(target.head(17000), predictions) # Calculate R-squared
  print(f'{model_name}: R-squared = {r2}') # Print R-squared for each model

In [ ]:
from sklearn.metrics import r2_score

# Assuming 'trained_models' is a dictionary of trained models
# And 'df_gmp_relaxed_scaled' is your feature data and 'target' your target variable

for model_name, model in trained_models.items():
  predictions = model.predict(unrelaxed.head(17000)) #Generate predictions
  r2 = r2_score(target.head(17000), predictions) # Calculate R-squared
  print(f'{model_name}: R-squared = {r2}') # Print R-squared for each model